# 2020 2C 2da Oportunidad

(***) Una compañía de logística de insumos médicos es responsable de toda la cadena de abastecimiento de estos a hospitales. Esto se ha vuelto crítico en los últimos meses para asistir
al personal médico en su lucha contra la pandemia. La logística se realiza sobre categorías de productos que se preparan en lotes (en pallets) que son entregados a las instituciones médicas.
Es por ello que cuentan con información en los siguientes archivos csv:

```
lote.csv (id_lote, fecha_programada_entrega, id_destino, descripcion, id_categoria_insumo, cantidad_de_unidades_insumo)
```
```
destinos.csv  (id_destino, nombre, atencion_critica, region, latitud, longitud)
```
```
categorias_insumos.csv (id_categoria_insumo, nombre, uso_esencial)
```

El primer archivo cuenta con información de los lotes de insumos a entrega, indicando la fecha programada de entrega (en formato ```“YYYY-mm-dd”```), cual es el destino al que se va a realizar la entrega (id_destino), una descripción del contenido del lote y cantidad de unidades del insumo que se encuentran en el lote.
Además se aporta información de la categoría a la que corresponde el insumo.

Por otro lado se tiene información sobre los distintos destinos en los que se deben entregar los insumos, su nombre, a que región pertenece, latitud y longitud.
Se cuenta también con un indicador booleano que indica si ese destino es un centro de atencion crítica.

En el último archivo, contamos con la información de las distintas categorías de insumos, indicando el nombre de la categoría y la columna booleana "uso_esencial" que nos indica que el producto de esa categoría de insumo es de uso esencial.

Se desea:

a) Listar los nombres de 10 destinos de la región de “CABA” que durante el mes de septiembre recibieron la mayor cantidad de lotes de insumos de uso esencial.

b) Listar el resumen del plan de entregas para la transportadora del dia de hoy.

Este debe tener el siguiente formato 
```(fecha_programada_entrega, nombre_destino, cantidad_de_lotes_a_entregar).```

A su vez debe estar ordenado de tal forma que primero estén listados aquellos destinos que son de atención crítica.


In [1]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [2]:
# armamos el df de lote.csv
lote_data = {
        'id_lote': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],
        'fecha_programada_entrega': ['2020-08-01','2020-09-01','2020-09-01','2020-08-01','2020-09-01','2020-10-01','2020-09-01','2020-09-01','2020-08-01','2020-08-01','2020-10-01','2020-09-01','2020-08-01','2020-08-01','2020-10-01'],
        'id_destino': [1,1,1,1,1,6,6,6,6,6,2,2,2,2,2],
        'descripcion': ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15'],
        'id_categoria_insumo': [1,1,1,1,1,2,2,2,2,2,2,3,3,3,3],
        'cantidad_de_unidades_insumo': [100,200,300,400,500,600,100,200,300,400,500,600,100,200,300],
}

df_lote = pd.DataFrame(lote_data, columns = lote_data.keys())

In [3]:
# armamos el df de destinos.csv
destinos_data = {
        'id_destino': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],
        'nombre': ["Destino 1","Destino 2","Destino 3","Destino 4","Destino 5","Destino 6","Destino 7","Destino 8","Destino 9","Destino 10","Destino 11","Destino 12","Destino 13","Destino 14", "Destino 15"],
        'atencion_critica': [1,1,1,1,1,0,0,0,0,0,0,0,0,0,0],
        'region': ['CABA','CABA','CABA','CABA','CABA','PBA1','PBA1','PBA1','PBA1','PBA1','PBA1','PBA2','PBA2','PBA2','PBA2'],
        'latitud': [1,1,1,1,1,0,0,0,0,0,0,0,0,1,1],
        'longitud': [1,1,1,1,1,0,0,0,0,0,0,0,0,1,1],
}

df_destinos = pd.DataFrame(destinos_data, columns = destinos_data.keys())

In [4]:
# armamos el data frame de datos de productos
categorias_insumos_data = {
        'id_categoria_insumo': [1,2,3],
        'nombre': ['Categoria 1','Categoria 2','Categoria 3'],
        'uso_esencial': [1,0,1],
}

df_categorias = pd.DataFrame(categorias_insumos_data, columns = categorias_insumos_data.keys())

In [5]:
df_lote.head(20)

,id_lote,fecha_programada_entrega,id_destino,descripcion,id_categoria_insumo,cantidad_de_unidades_insumo
0,1,2020-08-01,1,1,1,100
1,2,2020-09-01,1,2,1,200
2,3,2020-09-01,1,3,1,300
3,4,2020-08-01,1,4,1,400
4,5,2020-09-01,1,5,1,500
5,6,2020-10-01,6,6,2,600
6,7,2020-09-01,6,7,2,100
7,8,2020-09-01,6,8,2,200
8,9,2020-08-01,6,9,2,300
9,10,2020-08-01,6,10,2,400


In [6]:
df_destinos.head(6)

,id_destino,nombre,atencion_critica,region,latitud,longitud
0,1,Destino 1,1,CABA,1,1
1,2,Destino 2,1,CABA,1,1
2,3,Destino 3,1,CABA,1,1
3,4,Destino 4,1,CABA,1,1
4,5,Destino 5,1,CABA,1,1
5,6,Destino 6,0,PBA1,0,0


In [7]:
df_categorias

,id_categoria_insumo,nombre,uso_esencial
0,1,Categoria 1,1
1,2,Categoria 2,0
2,3,Categoria 3,1


In [8]:
#Primero filtro por fecha
df_lote['fecha_entrega_dt'] =  pd.to_datetime(df_lote['fecha_programada_entrega'])
df_lote_sept = df_lote[(df_lote['fecha_entrega_dt']>'2020-08') & (df_lote['fecha_entrega_dt']<'2020-10')]
df_lote_sept

,id_lote,fecha_programada_entrega,id_destino,descripcion,id_categoria_insumo,cantidad_de_unidades_insumo,fecha_entrega_dt
1,2,2020-09-01,1,2,1,200,2020-09-01
2,3,2020-09-01,1,3,1,300,2020-09-01
4,5,2020-09-01,1,5,1,500,2020-09-01
6,7,2020-09-01,6,7,2,100,2020-09-01
7,8,2020-09-01,6,8,2,200,2020-09-01
11,12,2020-09-01,2,12,3,600,2020-09-01


In [9]:
#destino_caba = df_destinos[df_destinos['region']=='CABA']
#destino_caba = destino_caba[['id_destino','nombre']]
destino_caba = df_destinos.loc[df_destinos['region']=='CABA',['id_destino','nombre']]
destino_caba

,id_destino,nombre
0,1,Destino 1
1,2,Destino 2
2,3,Destino 3
3,4,Destino 4
4,5,Destino 5


In [10]:
categorias_esenciales = df_categorias.loc[df_categorias['uso_esencial']==1,['id_categoria_insumo']]
categorias_esenciales

,id_categoria_insumo
0,1
2,3


In [11]:
lotes_esenciales_sep = pd.merge(df_lote_sept,categorias_esenciales,how='inner')
lotes_esenciales_sep = lotes_esenciales_sep[['id_lote','id_destino']]
lotes_esenciales_sep

,id_lote,id_destino
0,2,1
1,3,1
2,5,1
3,12,2


In [12]:
lotes_esenciales_sep_caba = pd.merge(lotes_esenciales_sep,destino_caba,how='inner')
lotes_esenciales_sep_caba

,id_lote,id_destino,nombre
0,2,1,Destino 1
1,3,1,Destino 1
2,5,1,Destino 1
3,12,2,Destino 2


In [14]:
lotes_esenciales_sep_por_destino = lotes_esenciales_sep_caba.groupby(['id_destino','nombre'])
res_fin = lotes_esenciales_sep_por_destino.count().reset_index().rename(columns={'id_lote':'cant_lotes'})
res_fin.nlargest(10,'cant_lotes')

,id_destino,nombre,cant_lotes
0,1,Destino 1,3
1,2,Destino 2,1


b) Listar el resumen del plan de entregas para la transportadora del dia de hoy.

Este debe tener el siguiente formato 
```(fecha_programada_entrega, nombre_destino, cantidad_de_lotes_a_entregar).```

A su vez debe estar ordenado de tal forma que primero estén listados aquellos destinos que son de atención crítica.
Supongamos que hoy es 01-09-2020

In [46]:
fecha_hoy = '2020-09-01'

In [48]:
lotes_hoy = df_lote.loc[df_lote['fecha_entrega_dt'] == fecha_hoy,['fecha_programada_entrega','id_destino']]
lotes_hoy

,fecha_programada_entrega,id_destino
1,2020-09-01,1
2,2020-09-01,1
4,2020-09-01,1
6,2020-09-01,6
7,2020-09-01,6
11,2020-09-01,2


In [51]:
destinos = df_destinos.iloc[:,0:3]
destinos

,id_destino,nombre,atencion_critica
0,1,Destino 1,1
1,2,Destino 2,1
2,3,Destino 3,1
3,4,Destino 4,1
4,5,Destino 5,1
5,6,Destino 6,0
6,7,Destino 7,0
7,8,Destino 8,0
8,9,Destino 9,0
9,10,Destino 10,0


In [58]:
lotes_hoy_nombres_destino = pd.merge(lotes_hoy,destinos,how='left',on='id_destino')
lotes_hoy_nombres_destino['cantidad_de_lotes_a_entregar'] = 1
group_hoy = lotes_hoy_nombres_destino.groupby(['fecha_programada_entrega','id_destino','nombre','atencion_critica']).count().reset_index()

In [64]:
group_hoy = group_hoy.sort_values(by='atencion_critica',ascending=False)
#fecha_programada_entrega, nombre_destino, cantidad_de_lotes_a_entregar)
resultado_final = group_hoy.rename(columns = {'nombre':'nombre_destino'})
resultado_final = resultado_final.loc[:,['fecha_programada_entrega', 'nombre_destino', 'cantidad_de_lotes_a_entregar']]
resultado_final

,fecha_programada_entrega,nombre_destino,cantidad_de_lotes_a_entregar
0,2020-09-01,Destino 1,3
1,2020-09-01,Destino 2,1
2,2020-09-01,Destino 6,2
